In [39]:
import pandas as pd

weather = pd.read_csv("lucknow_weather.csv", index_col="time")

In [40]:
weather

,tavg,tmin,tmax,prcp,Unnamed: 5,Unnamed: 6,Unnamed: 7
time,,,,,,,
01-01-1990,7.2,NaN,18.1,0.0,NaN,NaN,NaN
01-02-1990,10.5,NaN,17.2,0.0,NaN,NaN,NaN
01-03-1990,10.2,1.8,18.6,NaN,NaN,NaN,NaN
01-04-1990,9.1,NaN,19.3,0.0,NaN,NaN,NaN
01-05-1990,13.5,NaN,23.8,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
07-21-2022,27.4,25.1,33.1,27.3,NaN,NaN,NaN
07-22-2022,28.1,26.1,31.1,16.0,NaN,NaN,NaN
07-23-2022,30.3,26.2,34.7,11.9,NaN,NaN,NaN


In [43]:
null_pct = weather.apply(pd.isnull).sum()/weather.shape[0]
null_pct

tavg          0.011602
tmin          0.295527
tmax          0.130570
prcp          0.517236
Unnamed: 5    1.000000
Unnamed: 6    1.000000
Unnamed: 7    1.000000
dtype: float64

In [46]:
valid_columns = weather.columns[null_pct < .20]

In [47]:
valid_columns

Index(['tavg', 'tmax'], dtype='object')

In [48]:
weather = weather[valid_columns].copy()

In [49]:
weather.columns = weather.columns.str.lower()

In [50]:
weather

,tavg,tmax
time,,
01-01-1990,7.2,18.1
01-02-1990,10.5,17.2
01-03-1990,10.2,18.6
01-04-1990,9.1,19.3
01-05-1990,13.5,23.8
...,...,...
07-21-2022,27.4,33.1
07-22-2022,28.1,31.1
07-23-2022,30.3,34.7


In [51]:
weather = weather.ffill()

In [52]:
weather.apply(pd.isnull).sum()

tavg    0
tmax    0
dtype: int64

In [53]:
weather.dtypes

tavg    float64
tmax    float64
dtype: object

In [54]:
weather.index

Index(['01-01-1990', '01-02-1990', '01-03-1990', '01-04-1990', '01-05-1990',
       '01-06-1990', '01-07-1990', '01-08-1990', '01-09-1990', '01-10-1990',
       ...
       '07-16-2022', '07-17-2022', '07-18-2022', '07-19-2022', '07-20-2022',
       '07-21-2022', '07-22-2022', '07-23-2022', '07-24-2022', '07-25-2022'],
      dtype='object', name='time', length=11894)

In [55]:
weather.index = pd.to_datetime(weather.index)

In [56]:
weather.index.year.value_counts().sort_index()

time
1990    365
1991    365
1992    366
1993    365
1994    365
1995    365
1996    366
1997    365
1998    365
1999    365
2000    366
2001    365
2002    365
2003    365
2004    366
2005    365
2006    365
2007    365
2008    366
2009    365
2010    365
2011    365
2012    366
2013    365
2014    365
2015    365
2016    366
2017    365
2018    365
2019    365
2020    366
2021    365
2022    206
Name: count, dtype: int64

In [57]:
weather["target"] = weather.shift(-1)["tmax"]

In [58]:
weather

,tavg,tmax,target
time,,,
1990-01-01,7.2,18.1,17.2
1990-01-02,10.5,17.2,18.6
1990-01-03,10.2,18.6,19.3
1990-01-04,9.1,19.3,23.8
1990-01-05,13.5,23.8,21.4
...,...,...,...
2022-07-21,27.4,33.1,31.1
2022-07-22,28.1,31.1,34.7
2022-07-23,30.3,34.7,34.7


In [59]:
weather = weather.ffill()

In [60]:
weather

,tavg,tmax,target
time,,,
1990-01-01,7.2,18.1,17.2
1990-01-02,10.5,17.2,18.6
1990-01-03,10.2,18.6,19.3
1990-01-04,9.1,19.3,23.8
1990-01-05,13.5,23.8,21.4
...,...,...,...
2022-07-21,27.4,33.1,31.1
2022-07-22,28.1,31.1,34.7
2022-07-23,30.3,34.7,34.7


In [61]:
from sklearn.linear_model import Ridge

rr = Ridge(alpha=.1)

In [62]:
predictors = weather.columns[~weather.columns.isin(["target", "name", "station"])]

In [63]:
def backtest(weather, model, predictors, start=3650, step=90):
    all_predictions = []
    
    for i in range(start, weather.shape[0], step):
        train = weather.iloc[:i,:]
        test = weather.iloc[i:(i+step),:]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        combined["diff"] = (combined["prediction"] - combined["actual"]).abs()
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [64]:
predictions = backtest(weather, rr, predictors)

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mean_absolute_error(predictions["actual"], predictions["prediction"])

0.9702238872183475

In [66]:
predictions.sort_values("diff", ascending=False)

,actual,prediction,diff
time,,,
2007-06-13,28.3,37.872489,9.572489
2006-03-10,23.6,32.768667,9.168667
2010-01-08,12.4,21.511726,9.111726
2001-12-14,15.1,23.672577,8.572577
2019-04-16,31.6,40.137476,8.537476
...,...,...,...
2001-10-22,32.7,32.699342,0.000658
2014-03-02,25.4,25.400592,0.000592
2003-05-02,39.5,39.500296,0.000296


In [67]:
pd.Series(rr.coef_, index=predictors)

tavg    0.221823
tmax    0.744462
dtype: float64

In [69]:
def pct_diff(old, new):
    return (new - old) / old

def compute_rolling(weather, horizon, col):
    label = f"rolling_{horizon}_{col}"
    weather[label] = weather[col].rolling(horizon).mean()
    weather[f"{label}_pct"] = pct_diff(weather[label], weather[col])
    return weather
    
rolling_horizons = [3, 14]
for horizon in rolling_horizons:
    for col in ["tmax", "tavg"]:
        weather = compute_rolling(weather, horizon, col)

In [71]:
def expand_mean(df):
    return df.expanding(1).mean()

for col in ["tmax", "tavg"]:
    weather[f"month_avg_{col}"] = weather[col].groupby(weather.index.month, group_keys=False).apply(expand_mean)
    weather[f"day_avg_{col}"] = weather[col].groupby(weather.index.day_of_year, group_keys=False).apply(expand_mean)

In [72]:
weather = weather.iloc[14:,:]
weather = weather.fillna(0)

In [73]:
predictors = weather.columns[~weather.columns.isin(["target", "name", "station"])]

In [74]:
predictions = backtest(weather, rr, predictors)
mean_absolute_error(predictions["actual"], predictions["prediction"])

0.9048765011397446

In [75]:
predictions.sort_values("diff", ascending=False)

,actual,prediction,diff
time,,,
2010-01-08,12.4,21.275384,8.875384
2003-01-22,21.7,13.336615,8.363385
2006-03-10,23.6,31.931219,8.331219
2001-12-14,15.1,23.421374,8.321374
2007-06-13,28.3,36.611242,8.311242
...,...,...,...
2020-08-15,35.3,35.301115,0.001115
2001-12-08,28.6,28.599087,0.000913
2012-04-09,39.0,39.000868,0.000868
